In [1]:
import numpy as np
import tensorflow as tf

In [23]:
train_input = ['{0:020b}'.format(i) for i in range(2**10)]
np.random.shuffle(train_input)
train_input = [map(int,i) for i in train_input]
ti = []
for i in train_input:
    temp_list = []
    for j in i:
        temp_list.append([j])
    ti.append(np.array(temp_list))
train_input = np.array(ti)
train_input.shape

(1024, 20, 1)

In [25]:
train_output = []
for i in train_input:
    count = 0
    for j in i:
        if j[0] == 1:
            count += 1
    temp_list = ([0]*21)
    temp_list[count] = 1
    train_output.append(temp_list)
train_output = np.array(train_output)
train_output.shape

(1024, 21)

In [63]:

tf.reset_default_graph()

data = tf.placeholder(tf.float32,[None,20,1])
target = tf.placeholder(tf.float32,[None,21])
num_hidden = 24
cell = tf.nn.rnn_cell.LSTMCell(num_hidden,state_is_tuple=True)
val, state = tf.nn.dynamic_rnn(cell,data,dtype=tf.float32)
val = tf.transpose(val,[1,0,2])
last = tf.gather(val,int(val.get_shape()[0])-1)
weight = tf.Variable(
    tf.truncated_normal([num_hidden,int(target.get_shape()[1])])
    )
bias = tf.Variable(tf.constant(0.1, shape=[target.get_shape()[1]]))
prediction = tf.nn.softmax(tf.matmul(last,weight) + bias)
cross_entroy = -tf.reduce_sum(target * tf.log(tf.clip_by_value
                                             (prediction,1e-10,1.0)
                                             ))

opt = tf.train.AdamOptimizer().minimize(cross_entroy)

mistakes = tf.not_equal(tf.argmax(target, 1), tf.argmax(prediction, 1))
error = tf.reduce_mean(tf.cast(mistakes, tf.float32))

init = tf.global_variables_initializer()
sess = tf.InteractiveSession()
sess.run(init)
for i in range(1000):
    sess.run(opt,feed_dict={data : train_input,target : train_output})
    if i % 50 == 0:
        print sess.run(error,feed_dict={data : train_input,target : train_output})
sess.close()

0.758789
0.753906
0.514648
0.384766
0.276367
0.21875
0.173828
0.133789
0.09375
0.0634766
0.046875
0.0292969
0.0214844
0.0195312
0.015625
0.0136719
0.0136719
0.0136719
0.0126953
0.0117188


In [55]:
 tf.clip_by_value?